In [1]:
#Importing Packages
import networkx as nx
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
import os
from tqdm import tqdm

In [2]:
#Importing Signor Edges Database
P = nx.read_edgelist('parse_data/parse.biogrid.edgelist', encoding ='unicode_escape')
len(P.nodes)

22835

In [3]:
#Reading Nomenclature
nomenclature =['G','A','V','L','I','M','P','F','W','S','T','C','Y','N','Q','D','E','K','R','H']
hydro_score = [-0.4,1.8,4.2,3.8,4.5,1.9,-1.6,2.8,-0.9,-0.8,-0.7,2.5,-1.3,-3.5,-3.5,-3.5,-3.5,-3.9,-4.5,-3.2]

In [4]:
#Classifying Nomenclature in Gene Type
tiny =['A','C','G','S','T']
small = ['A','C','D','G','N','P','S','T','V']
aliphatic = ['I','L','V']
aromatic = ['F','H','W','Y']
nonpolar =['A','C','F','G','I','L','M','P','V','W','Y']
poscharged =['K','R','H']
negcharged = ['D','E']
charged = ['K','R','H','D','E']

In [5]:
#Identifying Protein Functions
GPCR = ['KW-0297']
ion = ['KW-1071','KW-0851','KW-0107','KW-0869','KW-0407','KW-0631','KW-0894']
kinase = ['KW-0418','KW-0723','KW-0829']
protease = ['KW-0031','KW-0064','KW-0121','KW-0224','KW-0482','KW-0645','KW-0720','KW-0788','KW-0888']
elem = GPCR+ion+kinase+protease

In [6]:
#Setting up Variables
error_nodes = []

indexes = []
gene_name = []
l = []
m=[]

aac_tiny = []
aac_small = []
aac_aliphatic = []
aac_aromatic = []
aac_nonpolar = []
aac_pos = []
aac_neg = []
aac_chr = []

G = []
A = []
V = []
L = []
I = []
M = []
Proline = []
F = []
W = []
S = []
T = []
C = []
Y = []
N = []
Q = []
D = []
E = []
K = []
R = []
H = []

gpcr_b = []
ionize_b = []
kin_b = []
prot_b = []

signalpep_b = []

N_count = []
O_count = []
serine_count = []
tyrosine_count = []
threonine_count = []
tsites = []

b_count = []
h_count = []
t_count = []

In [7]:
#Web Scraping for biological data
start_time = time.time()
traversed = 0
for node in tqdm(P.nodes()):
    link = 'https://www.uniprot.org/uniprot/' + str(node) +".xml"
    traversed += 1
    indexes.append(node)
    while True:
        try:
            db = requests.get(link)
            sp = BeautifulSoup(db.text, 'xml')
            
            #Parsing Protein Name
            name = str(sp.find('name',{'type': 'primary'}))
            name_parserA = name.find("\">")
            name_parserB = name.find("</")
            primary_name = name[name_parserA + len("\">") : name_parserB]
            gene_name.append(primary_name)
            #print('Name:', primary_name)
                               
            #Finding Protein Sequence and Information
            seq_info = sp.find_all('sequence')
            
            #Identifying length of sequence
            l_parserA = str(seq_info).find("length")
            l_parserB = str(seq_info).find("\" mass")
            length = str(seq_info)[l_parserA+len("length=\""):l_parserB]
            l.append(length)
            #print('Length:', length)
                               
            #Identifying mass of protein
            m_parserA = str(seq_info).find("mass")
            m_parserB = str(seq_info).find("\" modified")
            mass = str(seq_info)[m_parserA+len("mass=\""):m_parserB]
            m.append(mass)
            #print('Mass:', mass,'\n')
                               
            #Extracting Parsed Sequence
            s_parserA = str(seq_info).find("\">")
            s_parserB = str(seq_info).find("</")
            sequence = str(seq_info)[s_parserA+len("\">"):s_parserB]
            
            #Initializing Sequence Classifiers
            counts = [] 
            tinys = 0
            smalls = 0
            aliphatics = 0
            aromatics = 0
            nonpolars = 0
            poschargeds = 0
            negchargeds = 0
            chargeds = 0
            
            #Sequence Classifiers
            for x in nomenclature:
                c = sequence.count(x)
                counts.append(c)
                portion = c/int(length)
                if x in tiny:
                    tinys += portion
                if x in small:
                    smalls += portion
                if x in aliphatic:
                    aliphatics += portion
                if x in aromatic:
                    aromatics += portion
                if x in nonpolar:
                    nonpolars += portion
                if x in poscharged:
                    poschargeds += portion
                if x in negcharged:
                    negchargeds += portion
                if x in charged:
                    chargeds += portion
            
            #Saving Sequence Classifier Data
            aac_tiny.append(tinys)
            aac_small.append(smalls)
            aac_aliphatic.append(aliphatics)
            aac_aromatic.append(aromatics)
            aac_nonpolar.append(nonpolars)
            aac_pos.append(poschargeds)
            aac_neg.append(negchargeds)
            aac_chr.append(chargeds)
                               
            #Calculating hydroscores            
            hydro = 0
            sad = 0
            while sad<len(nomenclature):
                hydro += (hydro_score[sad]*counts[sad])
                sad += 1
            #print('Hydroscore:', hydro)
                               
            #Calculating Gene Frequencies
            density = np.array(counts)/int(length)
            G.append(density[0])
            A.append(density[1])
            V.append(density[2])
            L.append(density[3])
            I.append(density[4])
            M.append(density[5])
            Proline.append(density[6])
            F.append(density[7])
            W.append(density[8])
            S.append(density[9])
            T.append(density[10])
            C.append(density[11])
            Y.append(density[12])
            N.append(density[13])
            Q.append(density[14])
            D.append(density[15])
            E.append(density[16])
            K.append(density[17])
            R.append(density[18])
            H.append(density[19])
            #print(G[-1],H[-1])
                               
            #Classifying Protein Family Membership
            family = sp.find_all('keyword')
            gpcr = 0
            ionize = 0
            kin = 0
            prot = 0

            for keys in family:
                word = str(keys)[13:20]
                if word in GPCR:
                    gpcr = 1
                if word in ion:
                    ionize = 1
                if word in kinase:
                    kin = 1
                if word in protease:
                    prot = 1

            gpcr_b.append(gpcr)
            ionize_b.append(ionize)
            kin_b.append(kin)
            prot_b.append(prot)
            #print('GPCR:',gpcr)
            #print('Ionize:',ionize)
            #print('Kinase:',kin)
            #print('Protease:',prot)
                     
            #Determining Signal Peptide Membership                   
            signal_pep = 0
            signal = sp.find_all('feature', {'type' : 'signal peptide'})
            if len(signal)> 0:
                signal_pep = 1
            signalpep_b.append(signal_pep)
            #print('Signal Peptide:', signal_pep)
            
            #Counting Number of Alpha Helical Regions                   
            structure = sp.find_all('feature', {'type' : 'transmembrane region', 'description' :"Helical"})
            alpha_helical = len(structure)
            #print('Number of alphahelical transmembrane regions:', alpha_helical)
                               
            #Counting Number of Glycosylation Sites                   
            glyco_site = sp.find_all('feature', {'type' : 'glycosylation site'})
            N_glyc = 0
            O_glyc = 0
            glyco_l = 0

            for glycs in glyco_site:
                glyco_l +=1
                glyco_type = str(glycs)[22:54]
                if "N-linked" in glyco_type:
                    N_glyc += 1
                if "O-linked" in glyco_type:
                    O_glyc += 1


            if glyco_l != 0:
                #print("Normalized N-linked glycolysis:", N_glyc/glyco_l)
                N_count.append(N_glyc/glyco_l)
                #print("Normalized O-linked glycolysis:", O_glyc/glyco_l)
                O_count.append(O_glyc/glyco_l)
            else:
                N_count.append(0)
                O_count.append(0)
            
            phospo_site = sp.find_all('feature', {'type' : 'modified residue'})
            serine = 0
            threonine = 0
            tyrosine = 0
            phospho_l = 0

            #Counting Phosporilation Sites and Classifying them                   
            for phosps in phospo_site:
                phospho_l +=1
                phospho_type = str(phosps)[22:60]
                if "threonine" in phospho_type:
                    threonine += 1
                if "serine" in phospho_type:
                    serine += 1
                if "tyrosine" in phospho_type:
                    tyrosine += 1

            if phospho_l != 0:
                #print("Normalized Phosphorylation via serine:",serine/phospho_l)
                serine_count.append(serine/phospho_l)
                #print("Normalized Phosphorylation via threonine:",threonine/phospho_l)
                threonine_count.append(threonine/phospho_l)
                #print("Normalized Phosphorylation via tyrosine:",tyrosine/phospho_l)
                tyrosine_count.append(tyrosine/phospho_l)
            else:
                #print("Normalized Phosphorylation via serine: 0")
                serine_count.append(0)
                #print("Normalized Phosphorylation via threonine: 0")
                threonine_count.append(0)
                #print("Normalized Phosphorylation via tyrosine: 0")
                tyrosine_count.append(0)
                               
            total_sites = glyco_l + phospho_l
            #print("Total Sites:", total_sites)
            tsites.append(total_sites)
            
            #Beta Helix Turn Count
            beta = sp.find_all('feature', {'type' : 'strand'}) # DSSP secondary structure element codes : E and B
            beta_count = len(beta)
            helix = sp.find_all('feature', {'type' : 'helix'}) # DSSP secondary structure element codes : H, G, and I
            helix_count = len(helix)
            turn = sp.find_all('feature', {'type' : 'turn'}) # DSSP secondary structure element codes : T
            turn_count = len(turn)
            structure_count = beta_count + helix_count + turn_count
            if structure_count != 0 :
                # print("Normalized Beta count:", beta_count/structure_count)
                b_count.append(beta_count/structure_count)
                # print("Normalized Helix count:", helix_count/structure_count)
                h_count.append(helix_count/structure_count)
                # print("Normalized Turn count:", turn_count/structure_count)
                t_count.append(turn_count/structure_count)
            else:
                # print("Normalized Beta count:", beta_count/structure_count)
                b_count.append(0)
                # print("Normalized Helix count:", helix_count/structure_count)
                h_count.append(0)
                # print("Normalized Turn count:", turn_count/structure_count)
                t_count.append(0)
                               
        except requests.exceptions.Timeout:
            print('Timeout:', link)
            continue
        except requests.exceptions.TooManyRedirects:
            print('Redirected:', link)
            continue
        except requests.exceptions.ConnectionError:
            print('Retrying connection')
            continue
        except ValueError:
            print('Error found at node:', node, '... removing from biofeatures')
            error_nodes.append(node)
            del indexes[-1]
            del gene_name[-1]
            del l[-1]
            del m[-1]
            #break
        break

    if traversed % 50 == 0:
        check_time = time.time()-start_time
        print((traversed/len(P.nodes))*100,"% at ", check_time/60,"minutes.")

  0%|                                                         | 50/22835 [01:49<13:17:47,  2.10s/it]

0.21896211955331726 % at  1.8205041289329529 minutes.


  0%|▏                                                       | 100/22835 [03:28<11:30:39,  1.82s/it]

0.4379242391066345 % at  3.477032208442688 minutes.


  1%|▎                                                       | 150/22835 [04:55<11:03:06,  1.75s/it]

...

100%|██████████████████████████████████████████████████████▊| 22750/22835 [9:36:00<02:15,  1.59s/it]

99.62776439675936 % at  576.0017287453015 minutes.


100%|██████████████████████████████████████████████████████▉| 22800/22835 [9:37:17<01:02,  1.78s/it]

99.84672651631269 % at  577.2935526331265 minutes.


100%|███████████████████████████████████████████████████████| 22835/22835 [9:38:15<00:00,  1.52s/it]


In [8]:
biodata = pd.DataFrame()
biodata['UniprotID'] = indexes
biodata['Gene'] = gene_name
biodata['Length'] = l
biodata['Mass'] = m
biodata['Tiny Frequency'] = aac_tiny
biodata['Small Frequency'] = aac_small
biodata['Aliphatic Frequency'] = aac_aliphatic
biodata['Aromatic Frequency'] = aac_aromatic
biodata['Nonpolar Frequency'] = aac_nonpolar
biodata['Postive Charge Frequency'] = aac_pos
biodata['Negative Charge Frequency'] = aac_neg
biodata['Charged Frequency'] = aac_chr

biodata['G']=G
biodata['A']=A
biodata['V']=V
biodata['L']=L
biodata['I']=I
biodata['M']=M
biodata['P']=Proline
biodata['F']=F
biodata['W']=W
biodata['S']=S
biodata['T']=T
biodata['C']=C
biodata['Y']=Y
biodata['N']=N
biodata['Q']=Q
biodata['D']=D
biodata['E']=E
biodata['K']=K
biodata['R']=R
biodata['H']=H


biodata['GPCR'] = gpcr_b
biodata['Ionize'] = ionize_b
biodata['Kinease'] = kin_b
biodata['Protease'] = prot_b

biodata['SignalPeptide'] = signalpep_b
biodata['N-links'] = N_count
biodata['O-links'] = O_count
biodata['serine-links'] = serine_count
biodata['tyrosine-links'] = tyrosine_count
biodata['threonine-links'] = threonine_count
biodata['Binding Links'] = tsites
biodata['Helix'] = b_count
biodata['Beta'] = h_count
biodata['Turn'] = t_count

biodata.to_csv('parse_data/parse.biogrid.bio', sep='\t', encoding='unicode_escape')

In [9]:
with open('parse_data/parse.biogridv2.bio', 'w') as filehandle:
    filehandle.writelines('UniprotID\tGene\tLength\tMass\tTiny\tSmall\tAliphatic\tAromatic\tNonpolar\tPositive\tNegative\tCharged\tG\tA\tV\tL\tI\tM\tP\tF\tW\tS\tT\tC\tY\tN\tQ\tD\tE\tK\tR\tH\tGPCR\tIonize\tKinease\tProtease\tSignalPeptide\tNlink\tOlink\tSlink\tTylink\tThlink\tBindingLink\tHelix\tBeta\tTurn\n')
    for i in range(len(biodata['Gene'])):
        filehandle.writelines("%s\t" % indexes[i])
        filehandle.writelines("%s\t" % gene_name[i])
        filehandle.writelines("%s\t" % l[i])
        filehandle.writelines("%s\t" % m[i])
        filehandle.writelines("%s\t" % aac_tiny[i])
        filehandle.writelines("%s\t" % aac_small[i])
        filehandle.writelines("%s\t" % aac_aliphatic[i])
        filehandle.writelines("%s\t" % aac_aromatic[i])
        filehandle.writelines("%s\t" % aac_nonpolar[i])
        filehandle.writelines("%s\t" % aac_pos[i])
        filehandle.writelines("%s\t" % aac_neg[i])
        filehandle.writelines("%s\t" % aac_chr[i])

        filehandle.writelines("%s\t" % G[i])
        filehandle.writelines("%s\t" % A[i])
        filehandle.writelines("%s\t" % V[i])
        filehandle.writelines("%s\t" % L[i])
        filehandle.writelines("%s\t" % I[i])
        filehandle.writelines("%s\t" % M[i])
        filehandle.writelines("%s\t" % Proline[i])
        filehandle.writelines("%s\t" % F[i])
        filehandle.writelines("%s\t" % W[i])
        filehandle.writelines("%s\t" % S[i])
        filehandle.writelines("%s\t" % T[i])
        filehandle.writelines("%s\t" % C[i])
        filehandle.writelines("%s\t" % Y[i])
        filehandle.writelines("%s\t" % N[i])
        filehandle.writelines("%s\t" % Q[i])
        filehandle.writelines("%s\t" % D[i])
        filehandle.writelines("%s\t" % E[i])
        filehandle.writelines("%s\t" % K[i])
        filehandle.writelines("%s\t" % R[i])
        filehandle.writelines("%s\t" % H[i])


        filehandle.writelines("%s\t" % gpcr_b[i])
        filehandle.writelines("%s\t" % ionize_b[i])
        filehandle.writelines("%s\t" % kin_b[i])
        filehandle.writelines("%s\t" % prot_b[i])
        filehandle.writelines("%s\t" % signalpep_b[i])
        filehandle.writelines("%s\t" % N_count[i])
        filehandle.writelines("%s\t" % O_count[i])
        filehandle.writelines("%s\t" % serine_count[i])
        filehandle.writelines("%s\t" % tyrosine_count[i])
        filehandle.writelines("%s\t" % threonine_count[i])
        filehandle.writelines("%s\t" % tsites[i])
        filehandle.writelines("%s\t" % b_count[i])
        filehandle.writelines("%s\t" % h_count[i])
        filehandle.writelines("%s\n" % t_count[i])